In [2]:
import sympy as sp

def linear_approximation():
    x = sp.symbols('x')

    # Get a user-defined function and its derivative
    user_func = sp.sympify(input("Enter a function: "))
    derivative = sp.diff(user_func, x)

    # Choose a point for approximation
    approx_point = float(input("Enter the value: "))

    # Create functions for the user-defined function and its derivative
    func_eval = sp.lambdify(x, user_func)
    deriv_eval = sp.lambdify(x, derivative)

    # Find the nearest integer point for approximation
    rounded_value = round(func_eval(approx_point))
    nearest_point = round(approx_point)

    while func_eval(nearest_point) != rounded_value:
        nearest_point += 1 if func_eval(approx_point) - rounded_value < 0 else -1

    # Perform linear approximation using the formula: [f(a) + f'(a)(x - a)]
    result = func_eval(nearest_point) + deriv_eval(nearest_point) * (x - nearest_point)
    
    # Display the approximated result
    print("The nearest point is: ", nearest_point)
    print("Approximated result:", result.evalf(subs={x: approx_point}))

# Test the function
linear_approximation()

Enter a function: x**2
Enter the value: 27
The nearest point is:  27
Approximated result: 729.000000000000


In [ ]:
#Newton's Method:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np

def newton_method(func, initial_guess, a, b, tolerance=1e-6):
    x = sp.Symbol('x')
    f = sp.sympify(func)
    f_prime = sp.diff(f, x)
    
    if initial_guess < a or initial_guess > b:
        print("Initial guess is not within the interval [a, b]")
        return None
    
    x0 = initial_guess
    iterations = 0
    
    print(f"{'Iteration':<10}{'xn':<15}{'f(xn)':<20}{'f_prime(xn)':<20}{'f(xn)/f_prime(xn)':<25}{'xn - f(xn)/f_prime(xn)':<30}")
    
    x_values = []
    f_values = []
    
    while True:
        f_value = f.subs(x, x0)
        f_prime_value = f_prime.subs(x, x0)
        
        if f_prime_value == 0:
            print("Derivative is zero. Cannot continue.")
            return x_values, f_values, None
        
        x1 = x0 - f_value / f_prime_value
        
        x_values.append(x0)
        f_values.append(f_value)
        
        print(f"{iterations:<10}{x0:.6f}{f_value:.6f}{f_prime_value:.6f}{f_value / f_prime_value:.6f}{x1:.6f}")
        
        if abs(f_value) < tolerance:
            return x_values, f_values, x1
        
        # Check if the new guess is within the interval [a, b]
        if x1 < a or x1 > b:
            print("New guess is not within the interval [a, b]")
            return x_values, f_values, None
        
        x0 = x1
        iterations += 1
    
    return x_values, f_values, None

# Get user input for the function, initial guess, and the interval [a, b]
func = input("Enter the function (in terms of 'x'): ")
initial_guess = float(input("Enter the initial guess: "))
a = float(input("Enter the left endpoint of the interval [a]: "))
b = float(input("Enter the right endpoint of the interval [b]: "))

# Find the root using Newton's method
result = newton_method(func, initial_guess, a, b)
if result is not None:
    x_values, f_values, root = result
    if root is not None:
        print(f"Approximate root: {root:.6f} (found after {len(x_values)} iterations)")

    # Create a graph
    x_range = np.linspace(a, b, 1000)
    f_lambda = sp.lambdify(sp.Symbol('x'), sp.sympify(func), 'numpy')
    plt.plot(x_range, f_lambda(x_range), label=f'f(x) = {func}', color='blue')
    plt.plot(x_values, f_values, marker='o', linestyle='-', markersize=5, color='red', label='Newton Iterations')
    plt.axhline(0, color='black', linewidth=1)
    plt.axvline(0, color='black', linewidth=1)
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.title('Newton\'s Method')
    plt.axhline(0, color='black', linestyle='--', linewidth=0.5)
    if root is not None:
        plt.axvline(root, color='green', linestyle='--', linewidth=0.5, label='Root Approximation')
    plt.legend()
    plt.grid()
    plt.show()
else:
    print("Root not found within the specified interval.")


In [ ]:
#Trapezoid Method:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

def is_differentiable(func, a, b):
    try:
        func(a)
        func(b)
        func((a + b) / 2)
        return True
    except Exception:
        return False

def trapezoidal_rule(a, b, n, func):
    if not is_differentiable(func, a, b):
        print("The function is not differentiable over the interval [a, b].") #So, it will plot graph for every function except when function is not differentibale, 
        return None, None

    del_x = (b - a) / n
    x_values = [a + i * del_x for i in range(n + 1)]

    result = func(a) + func(b)
    formula_str = f"[{func(a)}"

    y_values = []

    for i in range(1, n):
        x_i = x_values[i]
        try:
            func_x_i = func(x_i)
            if func_x_i == 0:
                print(f"Warning: Division by zero at x = {x_i}. Skipping this point.")
                y_values.append(None)
                continue
            result += 2 * func_x_i
            formula_str += f" + 2f({x_i})"
            y_values.append(func_x_i)
        except (ZeroDivisionError, ValueError):
            print(f"Warning: Error at x = {x_i}. Skipping this point.")
            y_values.append(None)

    formula_str += f" + {func(b)}]*del_x"
    result *= del_x / 2

    print("Formula:", formula_str)
    print("Area under the curve:", result)

    return x_values, y_values

# Get user input
func_str = input("Enter the function: ")
a = float(input("Enter the lower limit of the integral (a): "))
b = float(input("Enter the upper limit of the integral (b): "))
n = int(input("Enter the number of subintervals: "))

# Convert the user input function string to a Python function
func = lambda x: eval(func_str)

# Calculate and print the x and y values
h = (b - a) / n
x_values = [a + i * h for i in range(n + 1)]
y_values = []

for i, x in enumerate(x_values):
    try:
        y = func(x)
        y_values.append(y)
        print(f"x = {x}, y = {y}")
    except (ZeroDivisionError, ValueError):
        y_values.append(None)
        print(f"Warning: Error at x = {x}. Skipping this point.")

# Calculate the area under the curve using the Trapezoidal method
x_values_no_nan, y_values_no_nan = trapezoidal_rule(a, b, n, func)

# Plot the function and the area under the curve
if x_values_no_nan is not None:
    x_values_no_nan = [x for x, y in zip(x_values_no_nan, y_values_no_nan) if y is not None]
    y_values_no_nan = [y for y in y_values_no_nan if y is not None]

    plt.plot(x_values_no_nan, y_values_no_nan, label='f(x)', marker='o')
    plt.axhline(0, color='black', linewidth=1)
    plt.axvline(0, color='black', linewidth=1)
    if x_values_no_nan:
        plt.fill_between(x_values_no_nan, y_values_no_nan, alpha=0.3, color='blue', label='Area under the curve')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend(loc='best')
    plt.title('Area under the curve = {:.4f}'.format(area))
    plt.grid(True)
    plt.show()


In [ ]:
#Midpoint Method
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

def is_differentiable(func, a, b):
    try:
        func(a)
        func(b)
        func((a + b) / 2)
        return True
    except Exception:
        return False

def midpoint_rule(a, b, n, func):
    if not is_differentiable(func, a, b):
        return None  # Return None to indicate that the function is not differentiable

    del_x = (b - a) / n
    x_values = [a + i * del_x for i in range(n + 1)]  # Corrected x_values calculation for n+1 intervals
    
    y_values = [func(x) for x in x_values]
    
    result = sum(y_values) * del_x
    
    return x_values, y_values, result

# Get user input
func_str = input("Enter the function: ")
a = float(input("Enter the lower limit of the integral (a): "))
b = float(input("Enter the upper limit of the integral (b): "))
n = int(input("Enter the number of subintervals: "))

# Convert the user input function string to a Python function
func = lambda x: eval(func_str)

# Calculate the area under the curve using the Midpoint Method
result = midpoint_rule(a, b, n, func)

if result is not None:
    x_values, y_values, area = result
    # Print the x_values and y_values
    print("x_values:", x_values)
    print("y_values:", y_values)

    # Plot the function and the area under the curve
    x_plot = [a + i * (b - a) / 1000 for i in range(1001)]  # Generate x values for plotting
    y_plot = [func(x) for x in x_plot]

    midpoints = [a + i * (b - a) / n for i in range(n)]  # Midpoints for filling
    y_midpoints = [func(x) for x in midpoints]

    # Add a dot at the midpoint
    plt.plot(x_plot, y_plot, label='f(x)')
    plt.axhline(0, color='black', linewidth=1)
    plt.axvline(0, color='black', linewidth=1)
    plt.fill_between(midpoints, y_midpoints, alpha=0.3, color='cyan', label='Area')
    plt.scatter(midpoints, y_midpoints, color='red', marker='o', label='Midpoint')  # Add a red dot for the midpoint
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend(loc='best')
    plt.title('Area under the curve = {:.4f}'.format(area))
    plt.grid(True)
    plt.show()
else:
    print("Function is not differentiable or division by zero error occurred. Cannot calculate the area.")


In [ ]:
#Simpson's Method [for even n]
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

def is_differentiable(func, a, b):
    try:
        func(a)
        func(b)
        func((a + b) / 2)
        return True
    except Exception:
        return False

def simpsons_rule(a, b, n, func):
    if n % 2 != 0:
        raise ValueError("The number of subintervals (n) must be even.")
    
    if not is_differentiable(func, a, b):
        print("The function is not differentiable over the interval [a, b].")
        return None
    
    del_x = (b - a) / n
    x_values = [a + i * del_x for i in range(n + 1)]

    result = func(a) + func(b)
    formula_str = f"[{func(a)}"
    
    for i in range(1, n):
        if i % 2 == 0:
            result += 2 * func(x_values[i])
            formula_str += f" + 2f(x{i})"
        else:
            result += 4 * func(x_values[i])
            formula_str += f" + 4f(x{i})"
        formula_str = formula_str.replace(f"x{i}", str(x_values[i]))
    
    formula_str += f" + {func(b)}]*del_x / 3"
    result *= del_x / 3
    
    return x_values, formula_str, result

# Get user input
func_str = input("Enter the function: ")
a = float(input("Enter the lower limit of the integral (a): "))
b = float(input("Enter the upper limit of the integral (b): "))
n = int(input("Enter the number of subintervals (even number): "))

# Convert the user input function string to a Python function
func = lambda x: eval(func_str)

# Calculate the x and y values
h = (b - a) / n
x_values = [a + i * h for i in range(n + 1)]
y_values = []

for x in x_values:
    try:
        y = func(x)
        y_values.append(y)
    except Exception:
        print("The function is not differentiable at x =", x)
        x_values = []
        y_values = []
        break
            
if x_values:
    print("x_values:", x_values)
    print("y_values:", y_values)

    # Calculate the area under the curve and get the formula
    x_values, formula, area = simpsons_rule(a, b, n, func)
    if x_values:
        print("Formula:", formula)

        # Plot the function and the area under the curve
        plt.plot(x_values, y_values, label='f(x)', marker='o')
        plt.axhline(0, color='black', linewidth=1)
        plt.axvline(0, color='black', linewidth=1)
        x_fill = np.linspace(a, b, 1000)
        y_fill = [func(x) for x in x_fill]
        plt.fill_between(x_fill, y_fill, alpha=0.3, color='blue', label='Area')
        plt.xlabel('x')
        plt.ylabel('f(x)')
        plt.legend(loc='best')
        plt.title('Area under the curve = {:.4f}'.format(area))
        plt.grid(True)
        plt.show()


In [ ]:
#Riemann sum Method
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

def is_differentiable(func, a, b):
    try:
        func.subs(x, a)
        func.subs(x, b)
        func.subs(x, (a + b) / 2)
        return True
    except Exception:
        return False

x = sp.symbols('x')

# Function to calculate the left Riemann sum
def left_riemann_sum(func, a, b, n):
    del_x = (b - a) / n
    sum_result = 0
    left_points = []
    differentiable = is_differentiable(func, a, b)
    if not differentiable:
        print("The function is not differentiable over the interval [{}, {}].")
        return None, None
    for i in range(n):
        x_val = a + i * del_x
        if x_val == a:
            left_points.append(x_val)
            continue
        try:
            sum_result += func.subs(x, x_val)
        except (ZeroDivisionError, ValueError, TypeError):
            print("The function is not differentiable over the interval [{}, {}].")
            return None, None
        left_points.append(x_val)
    return del_x * sum_result, left_points

# Function to calculate the right Riemann sum
def right_riemann_sum(func, a, b, n):
    del_x = (b - a) / n
    sum_result = 0
    right_points = []
    differentiable = is_differentiable(func, a, b)
    if not differentiable:
        print("The function is not differentiable over the interval [{}, {}].")
        return None, None
    for i in range(n):
        x_val = a + (i + 1) * del_x
        if x_val == a:
            right_points.append(x_val)
            continue
        try:
            sum_result += func.subs(x, x_val)
        except (ZeroDivisionError, ValueError, TypeError):
            print("The function is not differentiable over the interval [{}, {}].")
            return None, None
        right_points.append(x_val)
    return del_x * sum_result, right_points

if __name__ == "__main__":
    func_str = input("Enter a function as a string using 'x' as the variable: ")
    try:
        func = sp.sympify(func_str)
    except (sp.SympifyError, ValueError):
        print("Invalid input function. Please enter a valid mathematical expression.")
        exit(1)
    a = float(input("Enter the lower limit of the interval: "))
    b = float(input("Enter the upper limit of the interval: "))
    n = int(input("Enter the number of subintervals:"))

    differentiable = is_differentiable(func, a, b)
    if not differentiable:
        print("The function is not differentiable over the interval [{}, {}].")
        exit(1)

    left_sum, left_points = left_riemann_sum(func, a, b, n)
    if left_sum is not None:
        right_sum, right_points = right_riemann_sum(func, a, b, n)
        if right_sum is not None:
            area_difference = abs(left_sum - right_sum)
            print(f"Left Riemann sum: {left_sum}")
            print(f"Right Riemann sum: {right_sum}")
            print(f"Area difference between Left and Right Riemann sums: {area_difference}")

            if differentiable:  # Plot the graph only if the function is differentiable
                # Create a range of x values for plotting
                x_values = np.linspace(a, b, 100)

                # Calculate the corresponding y values for the function without evaluating it
                y_values = [None] * 100

                # Check if the function is differentiable at each point
                for i, val in enumerate(x_values):
                    try:
                        result = func.subs(x, val)
                        y_values[i] = float(result)
                    except (ZeroDivisionError, ValueError, TypeError):
                        print("The function is not differentiable over the interval [{}, {}].")
                        exit(1)

                # Create a figure and plot the function
                plt.figure(figsize=(10, 6))
                plt.plot(x_values, y_values, label="Function")

                # Plot left and right sums as vertical lines
                if left_points:
                    plt.vlines(left_points, ymin=0, ymax=[float(func.subs(x, x_val)) for x_val in left_points], color='red', linestyles='dotted', label='Left Riemann Sum')
                if right_points:
                    plt.vlines(right_points, ymin=0, ymax=[float(func.subs(x, x_val)) for x_val in right_points], color='blue', linestyles='dotted', label='Right Riemann Sum')

                plt.xlabel("x")
                plt.ylabel("f(x)")
                plt.title("Function and Riemann Sums")
                plt.legend()

                plt.show()
